In [2]:
suppressPackageStartupMessages(library(Seurat))
suppressPackageStartupMessages(library(scran))
suppressPackageStartupMessages(library(scater))
suppressPackageStartupMessages(library(emmeans))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(ggpubr))
suppressPackageStartupMessages(library(lmerTest))
suppressPackageStartupMessages(library(msigdbr))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(data.table))
setwd("/rds/general/user/emacdona/projects/covid19-transcriptome/live/sc_rnaseq/longitudinal_geneset/")

In [2]:
db<-c("hallmark","KEGG","Reactome")
model<-c("combined","wave_1","wave_2","vaccine_wave_2","vaccine_wave_2_mild_moderate_only")
cell_type<-c(
        "monocyte",
    "T_reg",
    "DC",
    "T_CD4",
    "NK",
    "T_CD8",
    "RBC",
    "B",
    "MAIT",
    "T_g_d",
    "B_ASC",
    "Platelet",
    "HSPC",
    "CD14mono",
    "CD16mono",
    "pDC",
    "T_CD4_Naive",
    "NK_2",
    "T_CD4_Helper",
    "T_CD8_CTL",
    "Int.mono",
    "T_CD4_CTL",
    "RBC_2",
    "B_ASC_dividing",
    "B_switched_mem",
    "NK_CD56_bright",
    "T_CD8_Naive",
    "DC2",
    "B_naive",
    "NKT",
    "T_CD8_Mem",
    "B_ASC_IgA",
    "Platelet_2",
    "HSPC_2",
    "B_ASC_IgM",
    "B_non-switched_mem",
    "DC3",
    "B_ASC_IgG",
    "DC1",
    "B_CD11c",
    "T_CD8_EMRA",
    "ASDC",
    "B_ASC_other"
    )

In [3]:
input<-data.frame()

for(d in db){
    for(m in model){
        for(cell in cell_type){
            input<-rbind(input,data.frame(db=d,model=m,cell_type=cell))
        }
    }
}

In [4]:
head(input)
dim(input)

,db,model,cell_type
,<chr>,<chr>,<chr>
1,hallmark,combined,monocyte
2,hallmark,combined,T_reg
3,hallmark,combined,DC
4,hallmark,combined,T_CD4
5,hallmark,combined,NK
6,hallmark,combined,T_CD8


[1] 645   3

In [6]:
df1<-apply(input[input$model=="combined",],1,function(x){
    print(paste0(x[[1]],"_",x[[2]],"_",x[[3]],"_sig.tsv"))
    if(file.exists(paste0(x[[1]],"_",x[[2]],"_",x[[3]],"_sig.tsv"))){
        res<-fread(paste0(x[[1]],"_",x[[2]],"_",x[[3]],"_sig.tsv"),data.table=FALSE)
        res$db<-x[[1]]
        res$model<-x[[2]]
        res$cell_type<-x[[3]]
        return(res)
   }
})

[1] "hallmark_combined_monocyte_sig.tsv"
[1] "hallmark_combined_T_reg_sig.tsv"
[1] "hallmark_combined_DC_sig.tsv"
[1] "hallmark_combined_T_CD4_sig.tsv"
[1] "hallmark_combined_NK_sig.tsv"
[1] "hallmark_combined_T_CD8_sig.tsv"
[1] "hallmark_combined_RBC_sig.tsv"
[1] "hallmark_combined_B_sig.tsv"
[1] "hallmark_combined_MAIT_sig.tsv"
[1] "hallmark_combined_T_g_d_sig.tsv"
[1] "hallmark_combined_B_ASC_sig.tsv"
[1] "hallmark_combined_Platelet_sig.tsv"
[1] "hallmark_combined_HSPC_sig.tsv"
[1] "hallmark_combined_CD14mono_sig.tsv"
[1] "hallmark_combined_CD16mono_sig.tsv"
[1] "hallmark_combined_pDC_sig.tsv"
[1] "hallmark_combined_T_CD4_Naive_sig.tsv"
[1] "hallmark_combined_NK_2_sig.tsv"
[1] "hallmark_combined_T_CD4_Helper_sig.tsv"
[1] "hallmark_combined_T_CD8_CTL_sig.tsv"
[1] "hallmark_combined_Int.mono_sig.tsv"
[1] "hallmark_combined_T_CD4_CTL_sig.tsv"
[1] "hallmark_combined_RBC_2_sig.tsv"
[1] "hallmark_combined_B_ASC_dividing_sig.tsv"
[1] "hallmark_combined_B_switched_mem_sig.tsv"
[1] "hallmark

In [7]:
df1<-do.call(rbind,df1)
head(df1)
dim(df1)

,pvals,qvals,singular,conv,pathway,db,model,cell_type
,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
1.1,0.003441796,0.08604489,0,0,HALLMARK_ADIPOGENESIS1,hallmark,combined,monocyte
1.2,0.040430699,0.30401664,0,0,HALLMARK_ALLOGRAFT_REJECTION2,hallmark,combined,monocyte
1.3,0.001995148,0.08604489,0,0,HALLMARK_INTERFERON_ALPHA_RESPONSE26,hallmark,combined,monocyte
1.4,0.009720215,0.16200358,0,0,HALLMARK_INTERFERON_GAMMA_RESPONSE27,hallmark,combined,monocyte
1.5,0.024961993,0.30401664,0,0,HALLMARK_KRAS_SIGNALING_UP29,hallmark,combined,monocyte
1.6,0.032065534,0.30401664,0,0,HALLMARK_MTORC1_SIGNALING31,hallmark,combined,monocyte


[1] 1788    8

In [9]:
df1<- df1 %>%
            mutate(level=case_when(cell_type%in%c("monocyte","T_reg","DC","T_CD4","NK","T_CD8","RBC","B","MAIT","T_g_d","B_ASC","Platelet","HSPC") ~ 1,
                              cell_type%in%c("CD14mono","CD16mono","pDC","T_CD4_Naive","NK_2","T_CD4_Helper","T_CD8_CTL","Int.mono","T_CD4_CTL","RBC_2","B_ASC_dividing","B_switched_mem","NK_CD56_bright","T_CD8_Naive","DC2","B_naive","NKT","T_CD8_Mem","B_ASC_IgA","Platelet_2","HSPC_2","B_ASC_IgM","B_non-switched_mem","DC3","B_ASC_IgG","DC1","B_CD11c","T_CD8_EMRA","ASDC","B_ASC_other") ~ 2))
                   

In [10]:
head(df1)

,pvals,qvals,singular,conv,pathway,db,model,cell_type,level
,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>
1.1,0.003441796,0.08604489,0,0,HALLMARK_ADIPOGENESIS1,hallmark,combined,monocyte,1
1.2,0.040430699,0.30401664,0,0,HALLMARK_ALLOGRAFT_REJECTION2,hallmark,combined,monocyte,1
1.3,0.001995148,0.08604489,0,0,HALLMARK_INTERFERON_ALPHA_RESPONSE26,hallmark,combined,monocyte,1
1.4,0.009720215,0.16200358,0,0,HALLMARK_INTERFERON_GAMMA_RESPONSE27,hallmark,combined,monocyte,1
1.5,0.024961993,0.30401664,0,0,HALLMARK_KRAS_SIGNALING_UP29,hallmark,combined,monocyte,1
1.6,0.032065534,0.30401664,0,0,HALLMARK_MTORC1_SIGNALING31,hallmark,combined,monocyte,1


In [16]:
sig<-df1[df1$qvals<0.05,]
head(sig)
dim(sig)

,pvals,qvals,singular,conv,pathway,db,model,cell_type,level
,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>
5.1,0.0017159446,0.04289861,0,0,HALLMARK_INTERFERON_ALPHA_RESPONSE26,hallmark,combined,NK,1
5.2,0.0008682177,0.04289861,0,0,HALLMARK_INTERFERON_GAMMA_RESPONSE27,hallmark,combined,NK,1
8.1,0.0001371959,0.00296015,0,0,HALLMARK_ALLOGRAFT_REJECTION2,hallmark,combined,B,1
8.2,0.0126126652,0.03503518,0,0,HALLMARK_ANDROGEN_RESPONSE3,hallmark,combined,B,1
8.3,0.0087203582,0.02954728,0,0,HALLMARK_APICAL_JUNCTION5,hallmark,combined,B,1
8.4,0.0117314163,0.03450417,0,0,HALLMARK_COAGULATION10,hallmark,combined,B,1


[1] 183   9

In [17]:
sig<-sig[order(sig$qvals),]
write.table(sig,"sig_longitudinal_genesets_combined.tsv",col.names=TRUE,row.names=FALSE,quote=FALSE,sep="\t")

In [23]:
################################
# Gather all the results
##################################

In [20]:
df1<-apply(input[input$model=="combined",],1,function(x){
    print(paste0(x[[1]],"_",x[[2]],"_",x[[3]],"_all.tsv"))
    if(file.exists(paste0(x[[1]],"_",x[[2]],"_",x[[3]],"_all.tsv"))){
        res<-fread(paste0(x[[1]],"_",x[[2]],"_",x[[3]],"_all.tsv"),data.table=FALSE)
        #print(head(res))
        res$db<-x[[1]]
        res$model<-x[[2]]
        res$cell_type<-x[[3]]
        return(res)
   }
})

[1] "hallmark_combined_monocyte_all.tsv"
[1] "hallmark_combined_T_reg_all.tsv"
[1] "hallmark_combined_DC_all.tsv"
[1] "hallmark_combined_T_CD4_all.tsv"
[1] "hallmark_combined_NK_all.tsv"
[1] "hallmark_combined_T_CD8_all.tsv"
[1] "hallmark_combined_RBC_all.tsv"
[1] "hallmark_combined_B_all.tsv"
[1] "hallmark_combined_MAIT_all.tsv"
[1] "hallmark_combined_T_g_d_all.tsv"
[1] "hallmark_combined_B_ASC_all.tsv"
[1] "hallmark_combined_Platelet_all.tsv"
[1] "hallmark_combined_HSPC_all.tsv"
[1] "hallmark_combined_CD14mono_all.tsv"
[1] "hallmark_combined_CD16mono_all.tsv"
[1] "hallmark_combined_pDC_all.tsv"
[1] "hallmark_combined_T_CD4_Naive_all.tsv"
[1] "hallmark_combined_NK_2_all.tsv"
[1] "hallmark_combined_T_CD4_Helper_all.tsv"
[1] "hallmark_combined_T_CD8_CTL_all.tsv"
[1] "hallmark_combined_Int.mono_all.tsv"
[1] "hallmark_combined_T_CD4_CTL_all.tsv"
[1] "hallmark_combined_RBC_2_all.tsv"
[1] "hallmark_combined_B_ASC_dividing_all.tsv"
[1] "hallmark_combined_B_switched_mem_all.tsv"
[1] "hallmark

In [21]:
df1<-do.call(rbind,df1)
head(df1)
dim(df1)

,pvals,qvals,singular,conv,pathway,db,model,cell_type
,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
1.1,0.003441796,0.08604489,0,0,HALLMARK_ADIPOGENESIS1,hallmark,combined,monocyte
1.2,0.040430699,0.30401664,0,0,HALLMARK_ALLOGRAFT_REJECTION2,hallmark,combined,monocyte
1.3,0.324820440,0.67223042,0,0,HALLMARK_ANDROGEN_RESPONSE3,hallmark,combined,monocyte
1.4,0.318525875,0.67223042,0,0,HALLMARK_ANGIOGENESIS4,hallmark,combined,monocyte
1.5,0.316004053,0.67223042,0,0,HALLMARK_APICAL_SURFACE6,hallmark,combined,monocyte
1.6,0.846289244,0.98033392,0,0,HALLMARK_APOPTOSIS7,hallmark,combined,monocyte


[1] 21198     8

In [22]:
df1<- df1 %>%
            mutate(level=case_when(cell_type%in%c("monocyte","T_reg","DC","T_CD4","NK","T_CD8","RBC","B","MAIT","T_g_d","B_ASC","Platelet","HSPC") ~ 1,
                              cell_type%in%c("CD14mono","CD16mono","pDC","T_CD4_Naive","NK_2","T_CD4_Helper","T_CD8_CTL","Int.mono","T_CD4_CTL","RBC_2","B_ASC_dividing","B_switched_mem","NK_CD56_bright","T_CD8_Naive","DC2","B_naive","NKT","T_CD8_Mem","B_ASC_IgA","Platelet_2","HSPC_2","B_ASC_IgM","B_non-switched_mem","DC3","B_ASC_IgG","DC1","B_CD11c","T_CD8_EMRA","ASDC","B_ASC_other") ~ 2))
                   

In [23]:
head(df1)

,pvals,qvals,singular,conv,pathway,db,model,cell_type,level
,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<dbl>
1.1,0.003441796,0.08604489,0,0,HALLMARK_ADIPOGENESIS1,hallmark,combined,monocyte,1
1.2,0.040430699,0.30401664,0,0,HALLMARK_ALLOGRAFT_REJECTION2,hallmark,combined,monocyte,1
1.3,0.324820440,0.67223042,0,0,HALLMARK_ANDROGEN_RESPONSE3,hallmark,combined,monocyte,1
1.4,0.318525875,0.67223042,0,0,HALLMARK_ANGIOGENESIS4,hallmark,combined,monocyte,1
1.5,0.316004053,0.67223042,0,0,HALLMARK_APICAL_SURFACE6,hallmark,combined,monocyte,1
1.6,0.846289244,0.98033392,0,0,HALLMARK_APOPTOSIS7,hallmark,combined,monocyte,1


In [28]:
write.table(df1,"all_longitudinal_genesets_combined.tsv",col.names=TRUE,row.names=FALSE,quote=FALSE,sep="\t")